# Imports

In [13]:
import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.max_columns', None)

# Datensätze

In [14]:
%store -r df_matches
%store -r df_players
%store -r df_rankings
%store -r df_matches_players
%store -r df_merge

In [15]:
df_Siegesbilanz = df_merge

In [16]:
df_players[df_players.name_last == "Becker"]

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
1413,101414,Boris,Becker,R,19671122.0,GER,190.0,Q76334
3793,103794,Benjamin,Becker,R,19810616.0,GER,178.0,Q77176
4571,104573,Jerome,Becker,R,19850622.0,GER,NaN,NaN
5688,105702,Richard,Becker,U,19910129.0,GER,NaN,Q16219372
7164,107179,Harvey,Becker,R,NaN,GBR,NaN,NaN
7616,107631,Marc,Becker,R,NaN,GER,NaN,NaN
10187,110202,Roger,Becker,R,19340206.0,GBR,NaN,Q1565958
10755,110770,Kai,Becker,R,NaN,GER,NaN,NaN
11537,111553,Ricky,Becker,U,19740319.0,USA,NaN,NaN
12889,112905,Jean,Becker,U,19150629.0,FRA,NaN,NaN


# Hilfsfunktionen

In [17]:
def fillna_0_astype_int64(dataframe:pd.DataFrame, columns:list=["Siege", "Niederlagen"]):

    for column in columns:
        dataframe[column] = dataframe[column].fillna(0).astype("int64")
    
    return dataframe

In [18]:
def siegesquote_funktion(dataframe:pd.DataFrame):

    dataframe["Siegesquote"] = dataframe["Siege"] / (dataframe["Siege"]+ dataframe["Niederlagen"])

    return dataframe

<br>

# Fragestellung 4:
### Wie entwickelt sich der sportliche Erfolg mit dem Alter? Gilt das für die Spielstatistiken gleichermaßen wie für die Platzierung auf der Rangliste?

In [19]:
#Datumsspalten in Datumsformat ändern
df_matches_players["tourney_date"] = pd.to_datetime(df_matches_players["tourney_date"], format='%Y%m%d')
df_matches_players["dob_winner"] = pd.to_datetime(df_matches_players["dob_winner"], format='%Y%m%d')
df_matches_players["dob_loser"] = pd.to_datetime(df_matches_players["dob_loser"], format='%Y%m%d')

<br>

## Zeitlicher Verlauf der Siegesquoten pro Spieler

In [20]:
#Tabelle mit den Siegen der Spieler pro Jahr
df_win = df_matches_players.groupby([df_matches_players["winner_id"],df_matches_players["tourney_date"].dt.year]).size().reset_index(name="Siege")

#Tabelle mit den Niederlagen der Spieler pro Jahr
df_lose = df_matches_players.groupby([df_matches_players["loser_id"],df_matches_players["tourney_date"].dt.year]).size().reset_index(name="Niederlagen")

#Tabllen zusammenfügen
df_merge = pd.merge(df_win, df_lose, how="outer", left_on=["winner_id", "tourney_date"], right_on=["loser_id", "tourney_date"])

#Spalten kombinieren, falls eine Spieler_id nur in einem der beiden DataFrames vorkommt, da bspw. in diesem Jahr noch kein Match gewonnen oder verloren wurde
df_merge['Spieler_id'] = df_merge['winner_id'].combine_first(df_merge['loser_id'])

#NaN-Werte entfernen und Datentyp zu int64 ändern
fillna_0_astype_int64(df_merge, ["Spieler_id", "Siege", "Niederlagen"])

#Spalte umbennennen
df_merge.rename(columns={"tourney_date": "Jahr"}, inplace=True)

#DataFrame sortieren
df_merge.sort_values(by=["Spieler_id", "Jahr"], ascending=[True, True], inplace=True)

#Siegesquote berechnen
siegesquote_funktion(df_merge)

#Vor- und Nachname hinzufügen
df_merge = pd.merge(df_merge, df_players, how="left", left_on="Spieler_id", right_on="player_id")
df_merge.rename(columns={"name_first":"Vorname", "name_last":"Nachname"}, inplace=True)
df_merge = df_merge[["Spieler_id", "Vorname", "Nachname", "Jahr", "Siege", "Niederlagen", "Siegesquote"]]

df_merge

,Spieler_id,Vorname,Nachname,Jahr,Siege,Niederlagen,Siegesquote
0,100001,Gardnar,Mulloy,1968,1,2,0.333333
1,100001,Gardnar,Mulloy,1969,2,2,0.500000
2,100001,Gardnar,Mulloy,1970,1,1,0.500000
3,100001,Gardnar,Mulloy,1977,0,1,0.000000
4,100002,Pancho,Segura,1968,3,7,0.300000
...,...,...,...,...,...,...,...
29263,211801,Jochen,Rosner,1968,0,1,0.000000
29264,211802,NaN,Grund,1968,0,1,0.000000
29265,211803,Heimo,Tschernatsch,1968,0,1,0.000000
29266,211804,Jesus,Oroquieta,1968,0,1,0.000000


<br>

## Zeitlicher Verlauf der Siegesquoten der "besten" 5 Spieler

Da die in unserem Beispiel die "besten" 5 Spieler extrem viele Matches gespielt haben und fast ihr gesamtes Leben nach dem Sport ausrichten, sind diese Spieler repräsentativ dafür, wie sich der sportliche Erfolg mit dem Alter verhält. 
Der Sport wird also immer auf das absolute Optimum getrieben, wodurch die Auswirkungen des Alters besser zu erkennen sind.

In [21]:
#Top 5 Spieler allgemein
df_Siegesbilanz.sort_values(by=["Siege", "Siegesquote"], ascending=[False, False])[0:5]

,Spieler_id,Vorname,Nachname,Siege,Niederlagen,Siegesquote
272,100284,Jimmy,Connors,1275,301,0.809010
1847,103819,Roger,Federer,1265,280,0.818770
2132,104745,Rafael,Nadal,1078,224,0.827957
565,100656,Ivan,Lendl,1075,242,0.816249
2185,104925,Novak,Djokovic,1045,207,0.834665


In [22]:
#Verlauf der Siegesquoten der Top 5 "besten" Spieler herausfiltern
df_top5 = df_merge[(df_merge.Spieler_id == 103819)|(df_merge.Spieler_id == 100284)|(df_merge.Spieler_id == 104745)|(df_merge.Spieler_id == 100656)|(df_merge.Spieler_id == 104925)]

In [23]:
fig = px.line(df_top5, x='Jahr', y='Siegesquote', color='Nachname', title='Siegesquote über die Jahre')

fig.update_layout(xaxis_title='Jahr', yaxis_title='Siegesquote')

fig.show()

Man kann deutlich erkennen, dass jeder der Spieler in seinen Anfangsjahren eher schlechte Siegesquoten verzeichnen konnte. Im Laufe der Jahre verbesserte sich diese und gelangte nach 5-10 Jahren an ihr Maximum. Dieses Level wurde dann mehr oder weniger für viele Jahre gehalten. 

Bei den Spielern Connors und Lendl nahm die Quote dann im weiteren Verlauf immer weiter ab, bis sie ab einem bestimmten Jahr gar keine Turniere mehr bestritten haben. Diesen Punkt kann man als Karriereende der Spieler interpretieren. Der Spieler Connors hat zudem noch eine auffällige Siegesquote und 0%, welche wir noch untersuchen werden.

Beim Spieler Federer ist auffällig, dass sein letztes und seit langer Zeit schlechtestes Turnier im Jahr 2021 stattfand. Auch das kann darauf hindeuten, dass seine Karriere wohl ein Ende gefunden hat.

Die Spieler Djokovic und Nadal scheinen noch "im Rennen" zu sein, da ihre Siegesquoten  auch 2022 immer noch auf einem relativ hohen Level sind und sie ihr erstes Turnier erst Jahre später als Federer gespielt haben.

<br>

<br>

### Was bedeuten die ungewöhnlichen Siegesquoten von 0% beim Spieler Connors gegen Ende seiner Karriere? 

In [24]:
df_merge[(df_merge.Spieler_id == 100284)&(df_merge.Siegesquote == 0)&(df_merge.Jahr > 1969)]

,Spieler_id,Vorname,Nachname,Jahr,Siege,Niederlagen,Siegesquote
2824,100284,Jimmy,Connors,1990,0,3,0.0
2830,100284,Jimmy,Connors,1996,0,1,0.0


In beiden Jahren hat Connors verhältnismäßig wenige Matches bestritten und davon keines gewonnen. Auf Grund dessen hat er eine Siegesquote von 0%.

<br>

## Gilt die Erkenntnis aus dem zeitlichen Verlauf der Siegesquoten auch für die Platzierung der Ranglisten?

In [25]:
df_rankings

,ranking_date,rank,player,points
0,20100104,1,103819,10550.0
1,20100104,2,104745,9205.0
2,20100104,3,104925,8310.0
3,20100104,4,104918,7030.0
4,20100104,5,105223,6785.0
...,...,...,...,...
3113256,20091228,1805,111421,1.0
3113257,20091228,1809,105648,1.0
3113258,20091228,1809,109144,1.0
3113259,20091228,1811,105675,1.0


In [26]:
#Datum in Datumsformat bringen
df_rankings["ranking_date"] = pd.to_datetime(df_rankings['ranking_date'], format='%Y%m%d')

#Sortieren nach Spieler ID und Datum 
df_filtered = df_rankings.sort_values(by=["player", "ranking_date"], ascending=[True, True])

#Vor- und Nachname hinzufügen
df_filtered = pd.merge(df_filtered, df_players, how="left", left_on="player", right_on="player_id")

#Spalten herausfiltern
df_filtered = df_filtered[["ranking_date", "rank", "player", "name_first", "name_last", "points"]]

#Zeilen herausfiltern
df_filtered = df_filtered[(df_filtered.player == 103819)|(df_filtered.player == 100284)|(df_filtered.player == 104745)|(df_filtered.player == 100656)|(df_filtered.player == 104925)]

df_filtered

,ranking_date,rank,player,name_first,name_last,points
20825,1973-08-27,10,100284,Jimmy,Connors,NaN
20826,1973-09-17,10,100284,Jimmy,Connors,NaN
20827,1973-10-01,5,100284,Jimmy,Connors,NaN
20828,1973-10-15,3,100284,Jimmy,Connors,NaN
20829,1973-11-05,4,100284,Jimmy,Connors,NaN
...,...,...,...,...,...,...
1662046,2022-11-28,5,104925,Novak,Djokovic,4820.0
1662047,2022-12-05,5,104925,Novak,Djokovic,4820.0
1662048,2022-12-12,5,104925,Novak,Djokovic,4820.0
1662049,2022-12-19,5,104925,Novak,Djokovic,4820.0


In [27]:
fig = px.line(df_filtered, x="ranking_date", y='rank', color='name_last', title='Ranglistenplatzierung über die Jahre', labels={"name_last": "Nachname"})

# Layout anpassen (optional)
fig.update_layout(xaxis_title='Jahr', yaxis_title='Rang')

# Diagramm anzeigen
fig.show()

Es ist auf den ersten Blick ein Zusammenhang zu der Grafik mit den Siegesquoten zu erkennen. Wenn die Siegesquote des Spielers signifikant steigt, sinkt parallel dazu der Rang. 

Bei Connors und Lendl steigt der Rang ab einem gewissen Alter über die Jahre an, bis sie schlussendlich wegen des Karriereendes aus der Ranglistenplatzierung entfernt wurden.

Bei der Verlaufskurve von Federer sieht man auch wieder, dass der Rang am Anfang der Karriere hoch ist und mit der Zeit immer niedriger wird. Dieses Level wird über viele Jahre gehalten. Im letzten Jahr in dem er ein Match gespielt hat, steigt sein Rang leicht an. 

Bei den Spielern Djokovic und Nadal berhält es sich so ähnlich wie bei Federer, nur das diese 2022 noch "im Rennen" waren und beide einen sehr niedrigen Rang aufweisen konnten.


<br>

## Schlusswort

Das Verhalten der Spielstatistiken ist nahezu identisch mit dem der Ranglistenplatzierungen. Der einzige Unterschied ist, dass die Platzierungsnummer sinkt, während die Siegesquote steigt.